<a href="https://colab.research.google.com/github/FindWhatYou/data_pipelining/blob/main/1_weather_and_flight_APIs_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# City list

In [ ]:
import pandas as pd

url = "https://drive.google.com/file/d/1gSBvbypiQwFRbdN0qVt4SULMMKCRULEc/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
city_all = pd.read_csv(path)

Only cities with more than 2 mil. population.

Use the `city_ascii` therefore there are only basic symbols.

In [ ]:
def city_targets(url,path,pop=200000,samp=1000):
    url = "https://drive.google.com/file/d/1gSBvbypiQwFRbdN0qVt4SULMMKCRULEc/view?usp=sharing"
    path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
    city_all = pd.read_csv(path)
    city_list = city_all[city_all["population"] > pop]
    city_targets = city_list[['city_ascii','iso3','lat','lng']].sample(samp).reset_index().copy()
    city_targets.drop(columns='index', inplace=True)
    cities_table = city_targets.reset_index().rename(columns={'index':'city_id'})
    return cities_table

In [ ]:
cities_table = city_targets(url,path,200000,5)

In [ ]:
cities_table

,city_id,city_ascii,iso3,lat,lng
0,0,Delmas,HTI,18.5500,-72.3000
1,1,Phoenix,USA,33.5722,-112.0892
2,2,Rome,ITA,41.8931,12.4828
3,3,Van,TUR,38.4942,43.3800
4,4,Douala,CMR,4.0500,9.7000


# APIs 1: Collect Weather Data

In [ ]:
import requests
import json
from datetime import datetime
import pytz
import os

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
def weather_in_city():
    # time of the API-Call
    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)

    # cities_table = city_targets(url,path,200000,city_number)

    API_key = os.environ['ENV_WEATHER_API']

    # building an dataframe
    weather_df ={
        'city_id': [],
        'city_name_api': [],
        'forecast_time': [],
        'outlook': [],
        'temperature': [],
        'feels_like': [],
        'wind_speed': [],
        'rain_prob': [],
        'time_stamp': []
        }

    # taking cities_table as input
    for i in range(len(cities_table)):
        country_code = cities_table.loc[i, "iso3"]
        city_name = cities_table.loc[i, "city_ascii"]
        lonfit = requests.get(f'http://api.openweathermap.org/data/2.5/forecast?q={city_name},{country_code}&appid={API_key}&units=metric')
        lonfit_json = lonfit.json()

        if lonfit_json['cod'] == '200':
            for j in range(40):
                weather_df['city_id'].append(cities_table.city_id[i])
                weather_df['city_name_api'].append(lonfit_json['city']['name'])
                weather_df['forecast_time'].append(lonfit_json['list'][j]['dt_txt'])
                weather_df['outlook'].append(lonfit_json['list'][j]['weather'][0]['description'])
                weather_df['temperature'].append(lonfit_json['list'][j]['main']['temp'])
                weather_df['feels_like'].append(lonfit_json['list'][j]['main']['feels_like'])
                weather_df['wind_speed'].append(lonfit_json['list'][j]['wind']['speed'])
                weather_df['rain_prob'].append(lonfit_json['list'][j]['pop'])
                weather_df['time_stamp'].append(now.strftime("%d-%m-%Y %H:%M:%S"))
        else:
            weather_df['city_id'].append(cities_table.city_id[i])
            weather_df['city_name_api'].append(city_name)
            weather_df['forecast_time'].append(None)
            weather_df['outlook'].append(lonfit_json['message'])
            weather_df['temperature'].append(None)
            weather_df['feels_like'].append(None)
            weather_df['wind_speed'].append(None)
            weather_df['rain_prob'].append(None)
            weather_df['time_stamp'].append(now.strftime("%d-%m-%Y %H:%M:%S"))

    weather_city_df = pd.DataFrame(weather_df)
    # change the datatype
    weather_city_df['time_stamp']= pd.to_datetime(weather_city_df.time_stamp)
    return weather_city_df

In [ ]:
weather_city_df = weather_in_city()

In [ ]:
weather_city_df

,city_id,city_name_api,forecast_time,outlook,temperature,feels_like,wind_speed,rain_prob,time_stamp
0,0,Delmas 73,2023-08-03 21:00:00,light rain,32.20,37.55,4.14,0.84,2023-03-08 20:30:22
1,0,Delmas 73,2023-08-04 00:00:00,moderate rain,28.60,32.91,2.04,0.96,2023-03-08 20:30:22
2,0,Delmas 73,2023-08-04 03:00:00,moderate rain,25.71,26.43,1.62,0.99,2023-03-08 20:30:22
3,0,Delmas 73,2023-08-04 06:00:00,overcast clouds,25.77,26.47,1.42,0.72,2023-03-08 20:30:22
4,0,Delmas 73,2023-08-04 09:00:00,overcast clouds,25.37,26.08,1.32,0.00,2023-03-08 20:30:22
...,...,...,...,...,...,...,...,...,...
195,4,Douala,2023-08-08 06:00:00,light rain,23.43,24.18,0.80,0.57,2023-03-08 20:30:22
196,4,Douala,2023-08-08 09:00:00,light rain,25.83,26.53,1.17,0.87,2023-03-08 20:30:22
197,4,Douala,2023-08-08 12:00:00,light rain,28.04,30.61,3.37,0.88,2023-03-08 20:30:22
198,4,Douala,2023-08-08 15:00:00,light rain,27.91,30.12,3.55,0.64,2023-03-08 20:30:22


Check for emmpty rows

In [ ]:
weather_city_df[weather_city_df['outlook'] == 0]

,city_id,city_name_api,forecast_time,outlook,temperature,feels_like,wind_speed,rain_prob,time_stamp




# APIs 2.1: Airports Of Cities

In [ ]:
headers = {
	"X-RapidAPI-Key": os.environ['ENV_AIRPORT_API3'],
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

In [ ]:
def retrive_aiport(city_id):
    lat = cities_table.loc[city_id, "lat"]
    lng = cities_table.loc[city_id, "lng"]
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
    querystring = {"lat":lat,"lon":lng,"radiusKm":"100","limit":"10"}
    response = requests.get(url, headers=headers, params=querystring)
    #airport_df =  pd.json_normalize(response.json())
    airport_df = pd.json_normalize(response.json()['items'])
    airport_df["city_id"] = cities_table.city_id[city_id]
    return airport_df

def city_airports(city_id):
    flight_data = pd.DataFrame()
    for i in range(city_id):
        airport_data = retrive_aiport(i)  # Assuming the function retrive_aiport(i) retrieves data for the ith airport
        flight_data = pd.concat([flight_data, airport_data], axis=0)
        flight_data_df = flight_data.rename(columns={'location.lat':'locationLat','location.lon':'locationLon'})
    return flight_data_df

In [ ]:
flight_data_df = city_airports(5).copy()

In [ ]:
flight_data_df

,icao,iata,name,shortName,municipalityName,countryCode,locationLat,locationLon,city_id
0,RPLL,MNL,Manila Ninoy Aquino,Ninoy Aquino,Manila,PH,14.508600,121.02000,0
1,RPLS,NSP,Cavite City Sangley Point Air Base,Sangley Point Air Base,Cavite City,PH,14.495400,120.90400,0
0,ZYCH,CNI,Changhai,Changhai,Changhai,CN,39.266666,122.66695,1
1,ZYTL,DLC,Dalian Zhoushuizi,Zhoushuizi,Dalian,CN,38.965700,121.53900,1
0,ZSYW,YIW,Yiwu,Yiwu,Yiwu,CN,29.344700,120.03200,2
0,ZYJL,JIL,Jilin City Jilin,Jilin,Jilin City,CN,44.002200,126.39600,3
0,ZGZJ,ZHA,Zhanjiang,Zhanjiang,Zhanjiang,CN,21.214400,110.35800,4


# APIs 2.2: Flight Information

In [ ]:
def flight_arrival():
    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)
    date = now.strftime("%Y-%m-%d")
    time_frame = ["00:00","11:59"]

    flight_data_df

    headers = {
        "X-RapidAPI-Key": os.environ['ENV_AIRPORT_API3'],
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"}


    flight_arrivals ={
        'icao': [],
        'arrival_time_local': [],
        'arrival_terminal': [],
        'departure_city': [],
        'departure_icao': [],
        'departure_time_local': [],
        'airline_flight_number': [],
        'time_stamp': []
         }



    for i in range(len(flight_data_df.icao)):
        icao = flight_data_df.icao.iloc[i]
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{date}T{time_frame[0]}/{date}T{time_frame[1]}"
        querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200:

            flights_json = response.json()

            for j in range(len(flights_json['arrivals'])):
                flight_arrivals['icao'].append(flight_data_df.icao.iloc[i]) # this in BS

                flight_arrivals['arrival_time_local'].append(flights_json['arrivals'][j]['arrival']['scheduledTimeLocal']) # j number ov arrivals in the city

                try:
                    flights_json['arrivals'][j]['departure']['terminal']
                except KeyError:
                    terminal = None
                    flight_arrivals['arrival_terminal'].append(None)
                else:
                    flight_arrivals['arrival_terminal'].append(flights_json['arrivals'][j]['departure']['terminal'])

                flight_arrivals['departure_city'].append(flights_json['arrivals'][j]['departure']['airport']['name'])

                try:
                    flights_json['arrivals'][j]['departure']['airport']['icao']
                except KeyError:
                    terminal = None
                    flight_arrivals['departure_icao'].append(None)
                else:
                   flight_arrivals['departure_icao'].append(flights_json['arrivals'][j]['departure']['airport']['icao'])

                try:
                    flights_json['arrivals'][j]['departure']['scheduledTimeLocal']
                except KeyError:
                    scheduledTimeLocal = None
                    flight_arrivals['departure_time_local'].append(None)
                else:
                   flight_arrivals['departure_time_local'].append(flights_json['arrivals'][j]['departure']['scheduledTimeLocal'])

                flight_arrivals['airline_flight_number'].append(flights_json['arrivals'][j]['number'])

                flight_arrivals['time_stamp'].append(now.strftime("%d-%m-%Y %H:%M:%S"))

        else:

            flight_arrivals['icao'].append(flight_data_df.icao.iloc[i])
            flight_arrivals['arrival_time_local'].append(None)
            flight_arrivals['arrival_terminal'].append(None)
            flight_arrivals['departure_city'].append(None)
            flight_arrivals['departure_icao'].append(None)
            flight_arrivals['departure_time_local'].append(None)
            flight_arrivals['airline_flight_number'].append(None)
            flight_arrivals['time_stamp'].append(now.strftime("%d-%m-%Y %H:%M:%S"))

    flight_arrivals_df = pd.DataFrame(flight_arrivals)

    flight_arrivals_reset = flight_arrivals_df.reset_index().copy()
    flight_arrivals_df = flight_arrivals_reset.rename(columns={"index": "flight_id"})

    flight_arrivals_df['time_stamp']           = pd.to_datetime(flight_arrivals_df.time_stamp)
    flight_arrivals_df['arrival_time_local']   = pd.to_datetime(flight_arrivals_df.arrival_time_local)
    flight_arrivals_df.dropna(subset=['departure_time_local'], inplace=True)
    flight_arrivals_df['departure_time_local'] = pd.to_datetime(flight_arrivals_df['departure_time_local'], utc=True)
    return flight_arrivals_df

In [ ]:
flight_arrival_df = flight_arrival().copy()

In [ ]:
flight_arrival_df

,flight_id,icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline_flight_number,time_stamp
1,1,RPLL,2023-08-04 00:30:00+08:00,1,Hong Kong,VHHH,2023-08-03 14:05:00+00:00,5J 115,2023-04-08 16:03:12
2,2,RPLL,2023-08-04 00:15:00+08:00,None,Bacolod City,RPVB,2023-08-03 14:50:00+00:00,5J 472,2023-04-08 16:03:12
3,3,RPLL,2023-08-04 00:50:00+08:00,2,Tokyo,RJAA,2023-08-03 12:00:00+00:00,5J 5057,2023-04-08 16:03:12
4,4,RPLL,2023-08-04 00:35:00+08:00,1,Lapu-Lapu City,RPVM,2023-08-03 15:00:00+00:00,5J 574,2023-04-08 16:03:12
5,5,RPLL,2023-08-04 00:25:00+08:00,None,Davao City,RPMD,2023-08-03 14:20:00+00:00,5J 974,2023-04-08 16:03:12
...,...,...,...,...,...,...,...,...,...
217,217,ZGZJ,2023-08-04 09:25:00+08:00,1,Shanghai,ZSPD,2023-08-03 22:25:00+00:00,FM 9357,2023-04-08 16:03:12
218,218,ZGZJ,2023-08-04 09:25:00+08:00,1,Haikou,ZJHK,2023-08-04 00:35:00+00:00,GS 6525,2023-04-08 16:03:12
219,219,ZGZJ,2023-08-04 09:50:00+08:00,None,Wuhan,ZHHH,2023-08-03 23:30:00+00:00,MU 2493,2023-04-08 16:03:12
221,221,ZGZJ,2023-08-04 10:10:00+08:00,None,Kunming,ZPPP,2023-08-04 00:20:00+00:00,MU 5875,2023-04-08 16:03:12


# Push the Data the SQL DB

In [ ]:
import sqlalchemy # install if needed

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

True

### AWS  Cred

In [ ]:
schema="gans_db"
host="wbs-project5-db.civ4ous8ttqf.eu-central-1.rds.amazonaws.com"
user="admin"
password=os.environ['MY_ENV_PWD']
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### Cities


In [ ]:
cities_table.to_sql('cities',
              if_exists='append',
              con=con,
              index=False)

5

### Weather

In [ ]:
weather_city_df.to_sql('weathers_table',
              if_exists='append',
              con=con,
              index=False)

200

### Airports

In [ ]:
flight_data_df.to_sql('cities_to_airport',
              if_exists='append',
              con=con,
              index=False)

7

### Flight Information

In [ ]:
flight_arrival_df.to_sql('flight_informations',
              if_exists='append',
              con=con,
              index=False)

184

## Testing

In [ ]:
pd.read_sql("show tables", con)

,Tables_in_gans_db
0,airport_names
1,cities
2,cities_to_airport
3,flight_informations
4,population
5,weathers_table


In [ ]:
pd.read_sql("SELECT * FROM cities", con)

,city_id,city_ascii,iso3,lat,lng
0,0,Santo Tomas,PHL,14.0833,121.183
1,1,Wafangdian,CHN,39.6271,121.997
2,2,Dongyang,CHN,29.2785,120.228
3,3,Shulan,CHN,44.4079,126.943
4,4,Wuchuan,CHN,21.4283,110.775


In [ ]:
pd.read_sql("SELECT * FROM weathers_table", con)

,city_id,city_name_api,forecast_time,outlook,temperature,feels_like,wind_speed,rain_prob,time_stamp
0,0,Delmas 73,2023-08-03 21:00:00,light rain,32.20,37.55,4.14,0.84,2023-03-08 20:30:22
1,0,Delmas 73,2023-08-04 00:00:00,moderate rain,28.60,32.91,2.04,0.96,2023-03-08 20:30:22
2,0,Delmas 73,2023-08-04 03:00:00,moderate rain,25.71,26.43,1.62,0.99,2023-03-08 20:30:22
3,0,Delmas 73,2023-08-04 06:00:00,overcast clouds,25.77,26.47,1.42,0.72,2023-03-08 20:30:22
4,0,Delmas 73,2023-08-04 09:00:00,overcast clouds,25.37,26.08,1.32,0.00,2023-03-08 20:30:22
...,...,...,...,...,...,...,...,...,...
195,4,Douala,2023-08-08 06:00:00,light rain,23.43,24.18,0.80,0.57,2023-03-08 20:30:22
196,4,Douala,2023-08-08 09:00:00,light rain,25.83,26.53,1.17,0.87,2023-03-08 20:30:22
197,4,Douala,2023-08-08 12:00:00,light rain,28.04,30.61,3.37,0.88,2023-03-08 20:30:22
198,4,Douala,2023-08-08 15:00:00,light rain,27.91,30.12,3.55,0.64,2023-03-08 20:30:22


In [ ]:
pd.read_sql("SELECT * FROM cities_to_airport", con)

,icao,iata,name,shortName,municipalityName,countryCode,locationLat,locationLon,city_id
0,RPLL,MNL,Manila Ninoy Aquino,Ninoy Aquino,Manila,P,14.5086,121.020,0
1,RPLS,NSP,Cavite City Sangley Point Air Base,Sangley Point Air Base,Cavite City,P,14.4954,120.904,0
2,ZGZJ,ZHA,Zhanjiang,Zhanjiang,Zhanjiang,C,21.2144,110.358,4
3,ZSYW,YIW,Yiwu,Yiwu,Yiwu,C,29.3447,120.032,2
4,ZYCH,CNI,Changhai,Changhai,Changhai,C,39.2667,122.667,1
5,ZYJL,JIL,Jilin City Jilin,Jilin,Jilin City,C,44.0022,126.396,3
6,ZYTL,DLC,Dalian Zhoushuizi,Zhoushuizi,Dalian,C,38.9657,121.539,1


In [ ]:
pd.read_sql("SELECT * FROM flight_informations", con)

,flight_id,icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline_flight_number,time_stamp
0,1,RPLL,2023-08-04 00:30:00,1.0,Hong Kong,VHHH,2023-08-03 14:05:00,5J 115,2023-04-08 16:03:12
1,2,RPLL,2023-08-04 00:15:00,NaN,Bacolod City,RPVB,2023-08-03 14:50:00,5J 472,2023-04-08 16:03:12
2,3,RPLL,2023-08-04 00:50:00,2.0,Tokyo,RJAA,2023-08-03 12:00:00,5J 5057,2023-04-08 16:03:12
3,4,RPLL,2023-08-04 00:35:00,1.0,Lapu-Lapu City,RPVM,2023-08-03 15:00:00,5J 574,2023-04-08 16:03:12
4,5,RPLL,2023-08-04 00:25:00,NaN,Davao City,RPMD,2023-08-03 14:20:00,5J 974,2023-04-08 16:03:12
...,...,...,...,...,...,...,...,...,...
179,217,ZGZJ,2023-08-04 09:25:00,1.0,Shanghai,ZSPD,2023-08-03 22:25:00,FM 9357,2023-04-08 16:03:12
180,218,ZGZJ,2023-08-04 09:25:00,1.0,Haikou,ZJHK,2023-08-04 00:35:00,GS 6525,2023-04-08 16:03:12
181,219,ZGZJ,2023-08-04 09:50:00,NaN,Wuhan,ZHHH,2023-08-03 23:30:00,MU 2493,2023-04-08 16:03:12
182,221,ZGZJ,2023-08-04 10:10:00,NaN,Kunming,ZPPP,2023-08-04 00:20:00,MU 5875,2023-04-08 16:03:12
